## Apply logistic regression to predict company bankruptcy 

### Dataset : 
The data were collected from the Taiwan Economic Journal for the years 1999 to 2009. Company bankruptcy was defined based on the business regulations of the Taiwan Stock Exchange.

Dataset : https://www.kaggle.com/fedesoriano/company-bankruptcy-prediction



In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression 
from pyspark.sql.functions import isnan, when, col, count

#### Create a spark session 

In [2]:
spark = SparkSession.builder.appName('Company bankruptcy prediction').getOrCreate()

#### Load the dataset 

In [3]:
df = spark.read.csv('/Users/ihebd/PycharmProjects/Spark-with-machine-learning-/datasets/company bankruptcy.csv', 
                   inferSchema= True, header= True)

Let's explore our dataset 

In [4]:
df.printSchema()

root
 |-- Bankrupt?: integer (nullable = true)
 |--  ROA(C) before interest and depreciation before interest: double (nullable = true)
 |--  ROA(A) before interest and % after tax: double (nullable = true)
 |--  ROA(B) before interest and depreciation after tax: double (nullable = true)
 |--  operating gross margin: double (nullable = true)
 |--  realized sales gross margin: double (nullable = true)
 |--  operating profit rate: double (nullable = true)
 |--  tax Pre-net interest rate: double (nullable = true)
 |--  after-tax net interest rate: double (nullable = true)
 |--  non-industry income and expenditure/revenue: double (nullable = true)
 |--  continuous interest rate (after tax): double (nullable = true)
 |--  operating expense rate: double (nullable = true)
 |--  research and development expense rate: double (nullable = true)
 |--  cash flow rate: double (nullable = true)
 |--  interest-bearing debt interest rate: double (nullable = true)
 |--  tax rate (A): double (nullable = t

In [5]:
df.take(5)

[Row(Bankrupt?=1,  ROA(C) before interest and depreciation before interest=0.370594257300249,  ROA(A) before interest and % after tax=0.424389446140427,  ROA(B) before interest and depreciation after tax=0.40574977247176,  operating gross margin=0.601457213277793,  realized sales gross margin=0.601457213277793,  operating profit rate=0.998969203197885,  tax Pre-net interest rate=0.796887145860514,  after-tax net interest rate=0.808809360876843,  non-industry income and expenditure/revenue=0.302646433889668,  continuous interest rate (after tax)=0.780984850207341,  operating expense rate=0.000125696868875964,  research and development expense rate=0.0,  cash flow rate=0.458143143520965,  interest-bearing debt interest rate=0.000725072507250725,  tax rate (A)=0.0,  per Net Share Value (B)=0.147949938898487,  Net Value Per Share (A)=0.147949938898487,  Net Value Per Share (C)=0.147949938898487,  Persistent EPS in the Last Four Seasons=0.16914058806845,  Cash Flow Per Share=0.3116644266817

In [6]:
df.count()

6819

#### Data wrangling 

Checking for Missing/Null values 

In [7]:
df.select([count(when(isnan(c), c)).alias(c) for c in df.columns]).show()

+---------+--------------------------------------------------------+---------------------------------------+--------------------------------------------------+-----------------------+----------------------------+----------------------+--------------------------+----------------------------+--------------------------------------------+-------------------------------------+-----------------------+--------------------------------------+---------------+------------------------------------+-------------+------------------------+------------------------+------------------------+----------------------------------------+--------------------+-------------------------+----------------------------------+---------------------------------------+----------------------------------------+-----------------------------+---------------------------------+-------------------------------+----------------------------------+------------------------+----------------------+-------------------------------------+

In [8]:
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---------+--------------------------------------------------------+---------------------------------------+--------------------------------------------------+-----------------------+----------------------------+----------------------+--------------------------+----------------------------+--------------------------------------------+-------------------------------------+-----------------------+--------------------------------------+---------------+------------------------------------+-------------+------------------------+------------------------+------------------------+----------------------------------------+--------------------+-------------------------+----------------------------------+---------------------------------------+----------------------------------------+-----------------------------+---------------------------------+-------------------------------+----------------------------------+------------------------+----------------------+-------------------------------------+

#### Logistic regression 

create feature vector 

In [9]:
df.columns

['Bankrupt?',
 ' ROA(C) before interest and depreciation before interest',
 ' ROA(A) before interest and % after tax',
 ' ROA(B) before interest and depreciation after tax',
 ' operating gross margin',
 ' realized sales gross margin',
 ' operating profit rate',
 ' tax Pre-net interest rate',
 ' after-tax net interest rate',
 ' non-industry income and expenditure/revenue',
 ' continuous interest rate (after tax)',
 ' operating expense rate',
 ' research and development expense rate',
 ' cash flow rate',
 ' interest-bearing debt interest rate',
 ' tax rate (A)',
 ' per Net Share Value (B)',
 ' Net Value Per Share (A)',
 ' Net Value Per Share (C)',
 ' Persistent EPS in the Last Four Seasons',
 ' Cash Flow Per Share',
 ' Revenue Per Share (Yuan)',
 ' Operating Profit Per Share (Yuan)',
 ' Per Share Net profit before tax (yuan)',
 ' realized sales gross profit growth rate',
 ' operating profit growth rate',
 ' after-tax net profit growth rate',
 ' regular net profit growth rate',
 ' continu

In [10]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler


In [11]:
asembler = VectorAssembler(inputCols=[' ROA(C) before interest and depreciation before interest',
 ' ROA(A) before interest and % after tax',
 ' ROA(B) before interest and depreciation after tax',
 ' operating gross margin',
 ' realized sales gross margin',
 ' operating profit rate',
 ' tax Pre-net interest rate',
 ' after-tax net interest rate',
 ' non-industry income and expenditure/revenue',
 ' continuous interest rate (after tax)',
 ' operating expense rate',
 ' research and development expense rate',
 ' cash flow rate',
 ' interest-bearing debt interest rate',
 ' tax rate (A)',
 ' per Net Share Value (B)',
 ' Net Value Per Share (A)',
 ' Net Value Per Share (C)',
 ' Persistent EPS in the Last Four Seasons',
 ' Cash Flow Per Share',
 ' Revenue Per Share (Yuan)',
 ' Operating Profit Per Share (Yuan)',
 ' Per Share Net profit before tax (yuan)',
 ' realized sales gross profit growth rate',
 ' operating profit growth rate',
 ' after-tax net profit growth rate',
 ' regular net profit growth rate',
 ' continuous net profit growth rate',
 ' total asset growth rate',
 ' net value growth rate',
 ' total asset return growth rate Ratio',
 ' cash reinvestment %',
 ' current ratio',
 ' quick ratio',
 ' interest expense ratio',
 ' total debt/total net worth',
 ' debt ratio %',
 ' net worth/assets',
 ' long-term fund suitability ratio (A)',
 ' borrowing dependency',
 ' contingent liabilities/net worth',
 ' Operating profit/paid-in capital',
 ' net profit before tax/paid-in capital',
 ' inventory and accounts receivable/net value',
 ' total asset turnover',
 ' accounts receivable turnover',
 ' average collection days',
 ' inventory turnover rate (times)',
 ' fixed assets Turnover frequency',
 ' net worth turnover rate (times)',
 ' revenue per person',
 ' operating profit per person',
 ' allocation rate per person',
 ' working capital to total assets',
 'Quick asset/Total asset',
 'current assets/total assets',
 'cash / total assets',
 'Quick asset /current liabilities',
 'cash / current liability',
 'current liability to assets',
 'operating funds to liability',
 'Inventory/working capital',
 'Inventory/current liability',
 'current liability / liability',
 'working capital/equity',
 'current liability/equity',
 'long-term liability to current assets',
 'Retained Earnings/Total assets',
 'total income / total expense',
 'total expense /assets',
 ' current asset turnover rate',
 ' quick asset turnover rate',
 ' working capitcal turnover rate',
 ' cash turnover rate',
 ' Cash flow to Sales',
 ' fix assets to assets',
 ' current liability to liability',
 'current liability to equity',
 'equity to long-term liability',
 'Cash flow to total assets',
 'cash flow to liability',
 'CFO to ASSETS',
 'cash flow to equity',
 'current liabilities to current assets',
 'one if total liabilities exceeds total assets zero otherwise',
 'net income to total assets',
 'total assets to GNP price',
 'No-credit interval',
 'Gross profit to Sales',
 "Net income to stockholder's Equity",
 'liability to equity',
 'Degree of financial leverage (DFL)',
 'Interest coverage ratio( Interest expense to EBIT )',
 'one if net income was negative for the last two year zero otherwise',
 'equity to liability'], outputCol='features')

In [12]:
datatransformed = asembler.transform(df)

In [13]:
datatransformed.select('features').show()

+--------------------+
|            features|
+--------------------+
|[0.37059425730024...|
|[0.46429093745429...|
|[0.42607127187637...|
|[0.39984400136498...|
|[0.46502218105591...|
|[0.38868034904694...|
|[0.39092282942524...|
|[0.50836055184517...|
|[0.48851947545459...|
|[0.49568566275045...|
|[0.48247452834787...|
|[0.44440111149027...|
|[0.49115195242041...|
|[0.47404085214254...|
|[0.50670306634816...|
|[0.51382050407058...|
|[0.48890947204212...|
|[0.53595281041290...|
|[0.50407058938234...|
|[0.48739823526544...|
+--------------------+
only showing top 20 rows



In [14]:
final_dataset = datatransformed.select('features', 'Bankrupt?')

our final dataset is composed of two columns one is the features and the other one is the label, that is the way spark handle data

In [15]:
final_dataset.show()

+--------------------+---------+
|            features|Bankrupt?|
+--------------------+---------+
|[0.37059425730024...|        1|
|[0.46429093745429...|        1|
|[0.42607127187637...|        1|
|[0.39984400136498...|        1|
|[0.46502218105591...|        1|
|[0.38868034904694...|        1|
|[0.39092282942524...|        0|
|[0.50836055184517...|        0|
|[0.48851947545459...|        0|
|[0.49568566275045...|        0|
|[0.48247452834787...|        0|
|[0.44440111149027...|        0|
|[0.49115195242041...|        0|
|[0.47404085214254...|        0|
|[0.50670306634816...|        0|
|[0.51382050407058...|        0|
|[0.48890947204212...|        0|
|[0.53595281041290...|        0|
|[0.50407058938234...|        0|
|[0.48739823526544...|        0|
+--------------------+---------+
only showing top 20 rows



let's split our dataset into training/test datasets

In [16]:
train_ds, test_ds = final_dataset.randomSplit([0.7, 0.3])

In [17]:
train_ds.describe().show()
test_ds.describe().show()

+-------+-------------------+
|summary|          Bankrupt?|
+-------+-------------------+
|  count|               4802|
|   mean|0.02936276551436901|
| stddev|0.16883876895612288|
|    min|                  0|
|    max|                  1|
+-------+-------------------+

+-------+--------------------+
|summary|           Bankrupt?|
+-------+--------------------+
|  count|                2017|
|   mean|0.039167079821517105|
| stddev|  0.1940404258195499|
|    min|                   0|
|    max|                   1|
+-------+--------------------+



Let's create our logistic regression model 

In [18]:
model = LogisticRegression(labelCol='Bankrupt?')

Model created ! Let's fit our model to the data 

In [19]:
linear_model = model.fit(train_ds)

In [20]:
training_summary = linear_model.summary

In [21]:
training_summary.predictions.describe().show()

+-------+-------------------+--------------------+
|summary|          Bankrupt?|          prediction|
+-------+-------------------+--------------------+
|  count|               4802|                4802|
|   mean|0.02936276551436901|0.010828821324448146|
| stddev|0.16883876895612288| 0.10350743481958365|
|    min|                0.0|                 0.0|
|    max|                1.0|                 1.0|
+-------+-------------------+--------------------+



Now that our model is created and fited, let's evaluate it 

In [22]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [23]:
test_result = linear_model.evaluate(test_ds)

In [24]:
test_result.predictions.show(200)

+--------------------+---------+--------------------+--------------------+----------+
|            features|Bankrupt?|       rawPrediction|         probability|prediction|
+--------------------+---------+--------------------+--------------------+----------+
|[0.0,0.0069232446...|        0|[-0.6204554980575...|[0.34967786259416...|       1.0|
|[0.20816067859406...|        1|[3.98081351168965...|[0.98167175200447...|       0.0|
|[0.27772631989470...|        1|[-12.430866647768...|[3.99338769407313...|       1.0|
|[0.28031004728708...|        1|[0.35174789129802...|[0.58704137373425...|       0.0|
|[0.31131477599571...|        1|[-4.9749622503774...|[0.00686137600285...|       1.0|
|[0.32496465655925...|        0|[3.28450120103874...|[0.96389326661998...|       0.0|
|[0.33627455759762...|        0|[1.34592396882308...|[0.79346244803858...|       0.0|
|[0.33895578413688...|        0|[4.70936914518516...|[0.99107000329686...|       0.0|
|[0.34100326622142...|        0|[12.8488877869851...|[

Evaluating model performance using UAC 

In [29]:
ev = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                  labelCol='Bankrupt?')

In [30]:
auc_eval = ev.evaluate(test_result.predictions)

In [32]:
auc_eval

0.5920987315645779